In [1]:
## import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from datenspende.utils import query_ch_df, query_pg_df
import datetime
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import os
import numpy as np
import glob


In [2]:
lc_u = np.load('uid_per_shb_fatigue.npy')
lc_u2=np.load('uid_per_shb_fatigue(1).npy')

In [3]:
path = os.getcwd()+'/user_df/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

df_lcu = []
demo_info = []
uid = []
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f, index_col = 0)
    for var in ['sleep', 'hr', 'steps']:
        if var not in df.columns:
            df[var] = np.nan
    df_lcu.append(df)
    demo_info.append(f.split("/")[-1].split('.')[0])
    uid.append(int(f.split("/")[-1].split('.')[1][1:])) 
      
sex = [re.split('(\d+)',demo_info[i])[0] for i in range(len(demo_info))]
age = [int(re.split('(\d+)',demo_info[i])[1]) for i in range(len(demo_info))] 

df_lcu.pop(31)
uid.pop(31)
sex.pop(31)
age.pop(31)
df_lcu.pop(67)
uid.pop(67)
sex.pop(67)
age.pop(67)

60

In [4]:
df_lcu_agg = []
for df in df_lcu:
    
    df['dt'] = pd.to_datetime(df['dt'])
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')
    # group by day and hourly - sum up steps and take mean of heart rate
    # per day 24 time bins with steps summed up and mean of heart rate
    
    #for du in range(len(df_lcu[0])): 
    d = df.copy()
    d['steps'] = d['steps'].fillna(0)
    d['steps'] = d['steps'] + 1
    
    d_agg = pd.DataFrame(columns = ['hr', 'd to max hr [%]', 'date', 'time','weekend', 'week_totest', 'phase',
       'steps', 'hros'])
    
    for day in d['date'].unique():
        d_day = d[d['date'] ==day]
        d_day_agg_m = d_day[['time', 'hr', 'd to max hr [%]']].groupby('time').mean().groupby(np.arange(96)//4).mean()
        d_day_agg_s = d_day[['time', 'steps']].groupby('time').sum().groupby(np.arange(96)//4).sum()
        hr = d_day_agg_m.loc[:,'hr']
        steps = d_day_agg_s.loc[:,'steps']
        
        
        d_day_agg_m['date'] = day
        d_day_agg_m['time'] = d_day_agg_m.index
        d_day_agg_m['weekend'] = d_day['weekend'].iloc[0]
        d_day_agg_m['week_totest'] = d_day['week_totest'].iloc[0]
        d_day_agg_m['phase'] = d_day['phase'].iloc[0]
        d_day_agg_m['steps'] = steps
        d_day_agg_m['hros'] = hr/steps
        
        
        d_agg = d_agg.append(d_day_agg_m)
    d_agg = d_agg.reset_index()
    df_lcu_agg.append(d_agg)


KeyboardInterrupt



In [ ]:
d_agg = df_lcu_agg[22]

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(16,8), sharey=True, sharex = True)
phases = ['pre', 'acute', 'sub-acute', 'post']
for ph in [0,1,2,3]:
  
    dfph = d_agg[d_agg['phase'] == ph]
    
    if ph == 0:
        n_a,bins = np.histogram(dfph[dfph['steps'] >4]['steps'])
    else: 
        n_a, _ = np.histogram(dfph[dfph['steps'] >4]['steps'], bins = bins)
    ax[ph].stairs(n_a/len(dfph[dfph['steps'] >4]['steps']), bins, fill=False)
 
    ax[ph].set_title(phases[ph])
    ax[ph].set_xlabel('steps');
ax[0].set_ylabel('relative frequency');

In [ ]:
n_sps = []
un_week = []
for d in df_lcu_agg:
    n_sp = []
    for w in d['week_totest'].unique():
        n_sp.append(len(d[(d['week_totest'] == w)&(d['steps'] > 3000)&(d['hr'] > 90)]))
    un_week.append(d['week_totest'].unique())
    n_sps.append(n_sp)

In [ ]:
i = 22

plt.scatter(un_week[i], n_sps[i])
plt.plot([0]*10, list(range(10)),c='r')
plt.xlabel('weeks to test')
plt.ylabel('# high activity');

In [ ]:
fig, axs = plt.subplots(29,4, figsize=(18, 14
                                      ), facecolor='w', edgecolor='k',sharex=True)
#fig.subplots_adjust(hspace = .5, wspace=.001)

axs = axs.ravel()

for d in range(len(n_sps)):

    axs[d].scatter(un_week[d], n_sps[d],s=.2)
    #axs[d].plot([0]*max(n_sps[d]), list(range(max(n_sps[d]))),c='r')
    axs[d].axvline(x=0,color='r')
    
     

In [ ]:
phases = ['pre', 'acute', 'sub-acute', 'post'
         ]

In [ ]:
d_agg = df_lcu_agg[79]

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(16,8),sharex = True, sharey = True)
for ph in [0,1,2,3]:
    dph = d_agg[d_agg['phase'] == ph]
    for dat in  dph['date'].unique():
        ax[ph].plot(dph[dph['date'] == dat]['steps'],dph[dph['date'] == dat]['hr'])
        x = dph[dph['date'] == dat]['steps'] -4
        y = dph[dph['date'] == dat]['hr']
        x= x.values
        y = y.values
        ax[ph].quiver(x[:-1], y[:-1], x[1:]-x[:-1], y[1:]-y[:-1],scale_units='xy',  angles='xy', scale=4,width=0.004)
        ax[ph].set_title(phases[ph])
        ax[ph].set_xlabel('steps')
        ax[ph].set_ylabel('hr');

In [ ]:
i = 80

d_agg = df_lcu_agg[i]
fig, ax = plt.subplots(1,4,figsize=(16,8),sharex = True, sharey = True)
for ph in [0,1,2,3]:
    dph = d_agg[d_agg['phase'] == ph]
    for dat in  dph['date'].unique():
        ax[ph].plot(dph[dph['date'] == dat]['steps'],dph[dph['date'] == dat]['hr'])
        x = dph[dph['date'] == dat]['steps'] -4
        y = dph[dph['date'] == dat]['hr']
        x= x.values
        y = y.values
        ax[ph].quiver(x[:-1], y[:-1], x[1:]-x[:-1], y[1:]-y[:-1],scale_units='xy',  angles='xy', scale=4,width=0.006)
        ax[ph].set_title(phases[ph])
        ax[ph].set_xlabel('steps')
        ax[ph].set_ylabel('hr');
        #ax[ph].set_xlim(0,40000)

In [ ]:
from matplotlib.lines import Line2D


In [ ]:
i = 22

d_agg = df_lcu_agg[i]
fig, ax = plt.subplots(1,1,figsize=(14,8),sharex = True, sharey = True)
cs = ['k', 'r', 'orange', 'green']
cl = [Line2D([i],[i], color=cs[i]) for i in range(len(cs))]
for ph in [0,1,2,3]:
    dph = d_agg[d_agg['phase'] == ph]
    for dat in  dph['date'].unique():
        ax.plot(dph[dph['date'] == dat]['steps'],dph[dph['date'] == dat]['hr'],c=cs[ph], alpha =0.1)
        x = dph[dph['date'] == dat]['steps'] -4
        y = dph[dph['date'] == dat]['hr']
        x= x.values
        y = y.values
        #ax.quiver(x[:-1], y[:-1], x[1:]-x[:-1], y[1:]-y[:-1],scale_units='xy',  angles='xy', scale=4,width=0.004)
        #ax.set_title(phases[ph])
        ax.set_xlabel('steps')
        ax.set_ylabel('hr');
    ax.legend(cl, phases)
    ax.set_xlim(0, max(d_agg['steps']))
    
    